In [ ]:
import numpy as np
import pandas as pd
from sergio import sergio

# Simulate Clean Data _ Steady-State Simulation

In [ ]:
sim = sergio(number_genes=100, number_bins = 9, number_sc = 300, noise_params = 1, decays=0.8, sampling_state=15, noise_type='dpd')
sim.build_graph(input_file_taregts ='data_sets/De-noised_100G_9T_300cPerT_1/Interaction_cID_1.txt', input_file_regs='data_sets/De-noised_100G_9T_300cPerT_1/Regs_cID_1.txt', shared_coop_state=2)
sim.simulate()
expr = sim.getExpressions()
expr_clean = np.concatenate(expr, axis = 1)

# Add Technical Noise _ Steady-State Simulations

In [ ]:
expr_O = sim.outlier_effect(expr, outlier_prob = 0.01, mean = 0.8, scale = 1)

expr_O_L = sim.lib_size_effect(expr_O, mean = 4.6, scale = 0.4)

binary_ind = sim.dropout_indicator(expr_O_L, shape = 6.5, percentile = 82)
expr_O_L_D = np.multiply(binary_ind, expr_O_L)

count_matrix = sim.convert_to_UMIcounts(expr_O_L_D)

count_matrix = np.concatenate(count_matrix, axis = 1)

# Simulate Clean Data _ differentiation Simulation

In [ ]:
df = pd.read_csv('data_sets/De-noised_100G_6T_300cPerT_dynamics_7/bMat_cID7.txt', sep='\t', header=None, index_col=None)
bMat = df.values

sim = sergio(number_genes=100, number_bins = 6, number_sc = 300, noise_params = 0.2, decays=0.8, sampling_state = 1, noise_params_splice = 0.07, noise_type='dpd', dynamics=True, bifurcation_matrix= bMat)
sim.build_graph(input_file_taregts ='data_sets/De-noised_100G_6T_300cPerT_dynamics_7/Interaction_cID_7.txt', input_file_regs='data_sets/De-noised_100G_6T_300cPerT_dynamics_7/Regs_cID_7.txt', shared_coop_state=2)
sim.simulate_dynamics()
exprU, exprS = sim.getExpressions_dynamics()
exprU_clean = np.concatenate(exprU_clean, axis = 1)
exprS_clean = np.concatenate(exprS_clean, axis = 1)

# Add Technical Noise _ differentiation Simulations

In [ ]:
exprU_O, exprS_O = sim.outlier_effect_dynamics(exprU, exprS, outlier_prob = 0.01, mean = 0.8, scale = 1)

exprU_O_L, exprS_O_L = sim.lib_size_effect_dynamics(exprU_O, exprS_O, mean = 4.6, scale = 0.4)

binary_indU = sim.dropout_indicator_dynamics(exprU_O_L, shape = 6.5, percentile = 82)
binary_indS = sim.dropout_indicator_dynamics(exprS_O_L, shape = 6.5, percentile = 82)
exprU_O_L_D = np.multiply(binary_indU, exprU_O_L)
exprS_O_L_D = np.multiply(binary_indS, exprS_O_L)

count_matrix_U = sim.convert_to_UMIcounts_dynamics(exprU_O_L_D)
count_matrix_S = sim.convert_to_UMIcounts_dynamics(exprS_O_L_D)

count_matrix_U = np.concatenate(count_matrix_U, axis = 1)
count_matrix_S = np.concatenate(count_matrix_S, axis = 1)